In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print 'Files in Drive:'
!ls drive/Work/TEMP/style_transfer

# Create a file in Drive.
# !echo "This newly created file will appear in your Drive file list." > drive/created.txt

In [0]:
print 'Files in Drive:'
!ls drive/Work/TEMP/

# Import keras functions

In [0]:
import numpy as np
import keras.backend as K
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Model

In [0]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

# Images to use - Plot them

In [0]:
content_image = 'drive/Work/TEMP/dog_image.jpg'
style_image = 'drive/Work/TEMP/Rembrandt_Dream_of_Joseph.jpg'
combined_image = 'drive/Work/TEMP/stylized_image.jpg'

In [0]:
height = 480
width = 640

In [0]:
content_img = image.load_img(content_image)
plt.imshow(content_img)
content_arr = np.expand_dims(np.asarray(content_img, dtype='float32'), axis=0)
print(content_arr.shape)

In [0]:
style_img = image.load_img(style_image)
plt.imshow(style_img)
style_arr = np.expand_dims(np.asarray(style_img, dtype='float32'), axis=0)
print(style_arr.shape)

# Mean center images

In [0]:
content_arr[:, :, :, 0] -= 103.939
content_arr[:, :, :, 1] -= 116.779
content_arr[:, :, :, 2] -= 123.68
content_arr = content_arr[:, :, :, ::-1]

style_arr[:, :, :, 0] -= 103.939
style_arr[:, :, :, 1] -= 116.779
style_arr[:, :, :, 2] -= 123.68
style_arr = style_arr[:, :, :, ::-1]

# Setup variables

In [0]:
content_image = K.variable(content_arr)
style_image = K.variable(style_arr)
combination_image = K.placeholder((1, height, width, 3))

input_tensor = K.concatenate([content_image, style_image, combination_image], 
                             axis=0)
print(input_tensor.shape)

# Download the VGG16 model

In [0]:
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# Get all the individual layers for later reference

In [0]:
layers = dict([(layer.name, layer.output) for layer in vgg16.layers])
layers

# Parameters for how the content and style will be used

In [0]:
content_weight = 0.025
style_weight = 5.0
total_variation_weight = 1.0

In [0]:
loss = K.variable(0.)

# Content loss

In [0]:
def content_loss(C, G):
    return K.sum(K.square(C-G))

In [0]:
layer_features = layers['block2_conv2']

In [0]:
content_features = layer_features[0, :, :, :]
generated_features = layer_features[2, :, :, :]

In [0]:
loss = loss + content_loss(content_features, generated_features)

# Style loss

In [0]:
def gram_matrix(a):
    return K.dot(a, K.transpose(a))

In [0]:
def style_loss(S, G):
    n_H, n_W, n_C = G.shape

    S = K.reshape(K.transpose(S), [n_C, n_H*n_W])
    G = K.reshape(K.transpose(G), [n_C, n_H*n_W])

    GS = gram_matrix(S)
    GG = gram_matrix(G)

    return K.sum(K.square(GS-GG)) / (4 * 3*3 * width**2 * height**2 )

In [0]:
feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']

for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, : , :]
    generated_features = layer_features[2, :, : , :]
    loss = loss + (style_weight/len(feature_layers)) * style_loss(style_features, generated_features)

# Total variation loss

In [0]:
def total_variation_loss(x):
    a = K.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = K.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    
    return K.sum(K.pow(a+b, 1.25))

loss += total_variation_weight * total_variation_loss(combination_image)

# Setup the optimization

In [0]:
grads = K.gradients(loss, generated_image)
outputs = [loss] + grads
f_outputs = K.function([generated_image], outputs)

In [0]:
def compute_loss_grads(x):
    x = x.reshape([1, height, width, 3])
    outs = f_outputs([x])
    loss_value = outs[0]
    grads_value = outs[1].flatten().astype('float64')
    return loss_value, grads_value

In [0]:
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grad_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        self.loss_value, self.grad_values = eval_loss_grads(x)
        return self.loss_value
    
    def grads(self, x):
        assert self.grad_values is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
    
evaluator = Evaluator()

In [0]:
# Start with a randomly initialized image
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

In [0]:
for i in range(10):
    print('Start of iteration', i)
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    

In [0]:
x = x.reshape((height, width, 3))
x = x[:, :, ::-1]
x[:, :, 0] += 103.939
x[:, :, 1] += 116.779
x[:, :, 2] += 123.68
x = np.clip(x, 0, 255).astype('uint8')

plt.imshow(x)